# Naive Bayes

In [1]:
import pandas as pd
import numpy as np
import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

In [2]:
GSPREADHSEET_DOWNLOAD_URL = (
    "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
)

FIUFIP_2021_1_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=FIUFIP_2021_1_GID))

### Llamado a funciones de preprocesamiento

In [3]:
df = preprocessing.remove_irrelevant_features(df)
df = preprocessing.missings_treatment(df)
df = preprocessing.one_hot_encodding(df)

### Separacion de datos de entrenamiento para los tres modelos a estudiar

In [4]:
X_train, X_val, y_train, y_val = preprocessing.dataset_split(df)

X_train_g = X_train[['ganancia_perdida_declarada_bolsa_argentina']]
X_val_g = X_val[['ganancia_perdida_declarada_bolsa_argentina']]

X_train_m = X_train.drop('ganancia_perdida_declarada_bolsa_argentina', axis='columns')
X_val_m = X_val.drop('ganancia_perdida_declarada_bolsa_argentina', axis='columns')

## Modelos

### Modelo 1
- se tratan todos los features como continuos

In [5]:
gnb = GaussianNB().fit(X_train, y_train)

y_pred = gnb.predict(X_val)
print(f"{np.sum(y_pred == y_val)} de {len(X_val)} predicciones correctas")

7633 de 9769 predicciones correctas


### Modelo 2, tratar todos los features como discretos 
- se descarta el feature ganancia_perdida_declarada_bolsa_argentina ya que toma demasiados valores y ademas tiene valores negativos que no son validos para el algoritmo

In [6]:
mnb = MultinomialNB().fit(X_train_m, y_train)

y_pred = mnb.predict(X_val_m)

print(f"{np.sum(y_pred == y_val)} de {len(X_val_m)} predicciones correctas")

7366 de 9769 predicciones correctas


### Modelo 3
- se usa una combinación de naive bayes multinomial y gaussiano

In [7]:
gnb = GaussianNB().fit(X_train_g, y_train)
mnb = MultinomialNB().fit(X_train_m, y_train)

X_train_proba_g = pd.DataFrame(gnb.predict_proba(X_train_g)).drop(0, axis='columns')
X_train_proba_m = pd.DataFrame(mnb.predict_proba(X_train_m)).drop(0, axis='columns')

# se construye el dataframe de probabilidades con los datos de entrenamiento
X_train_proba = pd.DataFrame()
X_train_proba['1_gaussian'] = X_train_proba_g[1]
X_train_proba['1_multinomial'] = X_train_proba_m[1]

# se aplica naive bayes gaussiano al dataframe de probabilidades
gnb_proba = GaussianNB().fit(X_train_proba, y_train)

# se aplica el modelo compuesto a los datos de validación y se obtiene la predicción
X_val_proba_g = pd.DataFrame(gnb.predict_proba(X_val_g)).drop(0, axis='columns')
X_val_proba_m = pd.DataFrame(mnb.predict_proba(X_val_m)).drop(0, axis='columns')
X_val_proba = pd.DataFrame()
X_val_proba['1_gaussian'] = X_val_proba_g[1]
X_val_proba['1_multinomial'] = X_val_proba_m[1]
y_pred = gnb_proba.predict(X_val_proba)

print(f"{np.sum(y_pred == y_val)} de {len(X_val_proba)} predicciones correctas")

7761 de 9769 predicciones correctas
